**İMPORT KUTUPHANE VE METRİKLER**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

warnings.filterwarnings("ignore")

DRİVE **BAĞLANTISI**

In [2]:
from google.colab import drive

# Google Drive'ı bağlayın
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

**OKUMA**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/web-page-phishing.csv" , sep=",", encoding='utf-8')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()/df.shape[0]

**VERİ DEĞER ARALIĞINI KUCULTME**

In [ ]:
df.columns.tolist()


In [ ]:
st = StandardScaler()

In [ ]:
df["url_length"] = st.fit_transform(df[["url_length"]])


In [ ]:
df.describe()

Koralasyon Matrixi

In [ ]:
plt.figure(figsize=(30,10))
ax = sns.heatmap(df.corr(numeric_only=True), annot=True)
plt.show()

In [ ]:
plt.style.use('ggplot')
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
X = df.drop('phishing', axis=1)
y = df['phishing']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Modeller

In [ ]:
model_list = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    KNeighborsClassifier(n_neighbors=3),
    RandomForestClassifier(n_estimators=100),
    GradientBoostingClassifier(n_estimators=100),
    AdaBoostClassifier(n_estimators=100),
    MLPClassifier(hidden_layer_sizes=(100,), max_iter=300),
    GaussianNB()
]

In [ ]:
results = []

In [ ]:
for i, model in enumerate(model_list):
    print(f"Training model - {str(model)}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    results.append({
        "Model": model,
        "Accuracy": accuracy,
        "Precision (0)": report['0']['precision'],
        "Recall (0)": report['0']['recall'],
        "F1-score (0)": report['0']['f1-score'],
        "Precision (1)": report['1']['precision'],
        "Recall (1)": report['1']['recall'],
        "F1-score (1)": report['1']['f1-score']
    })

In [ ]:
print("End of Training")
sns.set(font_scale=1.2)
plt.figure(figsize=(10, 6))
table = sns.heatmap(pd.DataFrame(results).set_index('Model'), annot=True, cmap="plasma", fmt=".2f", linewidths=.5, cbar=False)
plt.title("Classification Report for Different Models")
plt.show()

EN İYİ MODEL f1 Scoruna Göre

In [ ]:
for model in results:
    model["F1-score Avg"] = (model["F1-score (0)"] + model["F1-score (1)"]) / 2

sorted_models = sorted(results, key=lambda x: x["F1-score Avg"], reverse=True)
best_model = sorted_models[0]

print(f"Most suitable model: {str(best_model['Model'].__class__.__name__)} - F1 Score Avg: {str(best_model['F1-score Avg'])}")

In [ ]:
from joblib import dump, load

dump(best_model['Model'], 'best_model.joblib')

TEST CROSS_VALİD

In [ ]:
CrossValidResult = []

# Her model için çapraz doğrulama sonuçları (10x olarak belirledik)
for model in model_list:
    cv_results = cross_validate(model, X, y, cv=10,
                                scoring={'precision': 'precision',
                                         'recall': 'recall',
                                         'f1': 'f1',
                                         'accuracy': 'accuracy',
                                         'roc_auc': 'roc_auc',
                                         'average_precision': 'average_precision',
                                         'neg_brier_score': 'neg_brier_score'})

    model_results = {
        'Model': type(model).__name__,
        'Precision': cv_results['test_precision'].mean(),
        'Recall': cv_results['test_recall'].mean(),
        'F1': cv_results['test_f1'].mean(),
        'Accuracy': cv_results['test_accuracy'].mean(),
        'ROC AUC': cv_results['test_roc_auc'].mean(),
        'Average Precision': cv_results['test_average_precision'].mean(),
        'Neg Brier Score': cv_results['test_neg_brier_score'].mean()
    }

    CrossValidResult.append(model_results)

CrossValidResult_df = pd.DataFrame(CrossValidResult)
print(CrossValidResult_df)